## An SI model to a Tomato Spotted Wilt Virus experiment
In this example we will perform SI model inference using data on the spread of Tomato Spotted Wilt Virus (TSWV) in a greenhouse from Hughes et al. (1997). In this experiment, 520 plants regularly spaced in a 10m x 26m greenhouse were examined for pressence of TSWV once every two weeks. Plants were not removed after showing signs of infection by TSWV. The experiment concluded after 14 weeks, which saw a total of 327 individual plants infected.

### References
* Hughes G, McRoberts N, Madden LV, Nelson SC (1997). “Validating Mathematical Models of Plant-Disease Progress in Space and Time.” Mathematical Medicine and Biology: A Journal of the Institute of Mathematics and Its Applications, 14(2), 85– 112.

## Initialization
Inference with `Pathogen.jl` is able to utilize multiple processors, we will begin by adding processors to julia, and then loading `Pathogen.jl`

In [1]:
addprocs(3)
using Pathogen

Additionally we'll use:
* DataFrames.jl for individual level information
* Distributions.jl for specification of priors for Bayesian inference
* Plot.jl for visualization (using whichever visualization backend your prefer)
* CSV.jl for CSV file I/O

In [2]:
@everywhere using DataFrames, Distributions
using Plots, CSV

We'll set the seed for random number generation such that our results here are reproducible:

In [3]:
srand(5432)

MersenneTwister(UInt32[0x00001538], Base.dSFMT.DSFMT_state(Int32[-1744756614, 1072761743, -2028083806, 1073624598, -581637934, 1073626379, 327696063, 1073382742, 738168642, 1073600735  …  -595993142, 1072726157, 913087430, 1073114622, 2001550298, -624279679, 872770575, 1763491762, 382, 0]), [1.97539, 1.06875, 1.50308, 1.82475, 1.06683, 1.34572, 1.9635, 1.70965, 1.23669, 1.42295  …  1.76843, 1.19712, 1.11801, 1.09259, 1.33145, 1.10836, 1.82953, 1.69841, 1.71411, 1.21945], 382)

## TSWV Data
We'll import the TSWV, provided as csv files. 

The first CSV file contains X, Y locations in metres for each individual plant in the study. 

The second CSV file contains records of the first day in which each individual was observed as being infected by TSWV. `NaN` indicates that no signs of infection were observed within the 14 week study period.

In [4]:
# Use CSV.jl for DataFrames I/O
#
# We know the types of the columns, so we'll manually specify those. 
# * Individual IDs are `Int64`
# * X,Y coordinates are `Float64`s
pop = CSV.read(Pkg.dir("Pathogen")*"/examples/02_TSWV_locations.csv", types=[Int64; Float64; Float64])

# Use julia's included CSV interface for simple vector of observation times
raw_observations = readcsv(Pkg.dir("Pathogen")*"/examples/02_TSWV_infection_observations.csv")[:]

# Create an `EventObservations` object with `Pathogen.jl`
obs = EventObservations{SI}(raw_observations)

Pathogen.SI model observations (n=520)

We will now formulate our `SI` individual level model. For our example, this model will be quite simple as we have a contained artifical environment which limits exogeneous transmissions, and we do not have individual level risk factors to consider beyond basic location data. We will use some common functions which have been prewritten in our examples folder.



In [5]:
@everywhere include(Pkg.dir("Pathogen")*"/examples/risk_functions.jl")

rf = RiskFunctions{SI}(_zero, # sparks function - we will assume no exogenous transmissions and set this to zero
                       _one, # susceptibility function - we do not have individual level risk factor information to explore here, so will set to a constant 1
                       _powerlaw_w_intercept, # transmissability function - we will use a powerlaw (with intercept) kernel. This provides a spatial and non-spatial component to infection transmissions. This has 3 parameters.
                       _one) # infectivity function - we do not have individual level risk factor information to explore here, so will set to a constant 1

Pathogen.SI model risk functions

In [6]:
rpriors = RiskPriors{SI}(UnivariateDistribution[], # empty `UnivariateDistribution` vector for all parameter-less functions
                         UnivariateDistribution[], 
                         [Gamma(10.0, 10.0); Gamma(10.0, 10.0); Gamma(1.0, 1.0)], # Relatively uninformative priors with appropriate support
                         UnivariateDistribution[])

Pathogen.SI model risk function priors

We provide some bounds to event times in comparision to observation times. Actual onset of infectiousness in this study could have occurred any time between plant examinations. This means the observation delay could be up to 14.0 days.

In [7]:
ee = EventExtents{SI}(14.0)

Pathogen.SI model event extents

In [8]:
mcmc = MCMC(obs, ee, pop, rf, rpriors)
start!(mcmc, 3, attempts = 50, debug_level = 2)

start!: 3 Markov chain initialization Futures successfully created
start!: 3 Markov chains successfully initialized


Pathogen.SI model MCMC with 3 chains

In [ ]:
iterate!(mcmc, 1000, diagm([0.1; 0.1; 0.01]), 1.0, debug_level = 2)